In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

def load_and_combine_results(dataset_ids=[1, 2, 3, 4], missing_rates=[10, 20, 30, 40]):
    """Carrega todos os arquivos de resultados e combina em um único DataFrame."""
    all_files = []
    for dataset in dataset_ids:
        for rate in missing_rates:
            file_pattern = f"results/results_dataset{dataset}_missing{rate}.csv"
            matching_files = glob.glob(file_pattern)
            if not matching_files:
                print(f"Aviso: Arquivo não encontrado - {file_pattern}")
                continue
            all_files.extend(matching_files)
    
    if not all_files:
        raise FileNotFoundError("Nenhum arquivo de resultados encontrado!")
    
    return pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

def analyze_results(df):
    """Realiza análise comparativa dos resultados."""
    # Configuração de estilo para os gráficos
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 8))
    
    # Converter missing_rate para porcentagem para facilitar leitura
    df['missing_rate'] = df['missing_rate'] * 100
    
    # 1. Comparação geral dos métodos
    print("\n=== Desempenho Médio por Método ===")
    print(df.groupby('method')[['rmse', 'nrmse']].mean().sort_values('nrmse'))
    
    # Gráfico de desempenho médio
    plt.figure(figsize=(10, 6))
    sns.barplot(x='method', y='nrmse', data=df, estimator=np.mean, ci=None)
    plt.title('NRMSE Médio por Método de Imputação')
    plt.ylabel('NRMSE Médio')
    plt.xlabel('Método')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('average_performance.png')
    plt.close()
    
    # 2. Desempenho por taxa de missing
    print("\n=== Desempenho por Taxa de Dados Faltantes ===")
    print(df.groupby(['method', 'missing_rate'])[['rmse', 'nrmse']].mean().unstack())
    
    plt.figure(figsize=(12, 6))
    sns.lineplot(x='missing_rate', y='nrmse', hue='method', data=df, 
                 estimator=np.mean, ci=None, marker='o')
    plt.title('Desempenho por Taxa de Dados Faltantes')
    plt.ylabel('NRMSE Médio')
    plt.xlabel('Taxa de Dados Faltantes (%)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('performance_by_missing_rate.png')
    plt.close()
    
    # 3. Comparação SoftImpute vs IterativeSVD por configuração
    matrix_methods = df[df['method'].isin(['SoftImpute', 'IterativeSVD'])]
    if not matrix_methods.empty:
        print("\n=== Comparação SoftImpute vs IterativeSVD ===")
        
        # Por tamanho de matriz
        plt.figure(figsize=(12, 6))
        sns.boxplot(x='n_lines', y='nrmse', hue='method', data=matrix_methods)
        plt.title('Desempenho por Tamanho da Matriz Temporal')
        plt.ylabel('NRMSE')
        plt.xlabel('Número de Linhas na Matriz')
        plt.tight_layout()
        plt.savefig('matrix_methods_by_matrix_size.png')
        plt.close()
        
        # Melhores parâmetros para cada método
        print("\nMelhores parâmetros para SoftImpute:")
        best_soft = matrix_methods[matrix_methods['method'] == 'SoftImpute']
        best_soft_params = best_soft.loc[best_soft.groupby(['missing_rate', 'n_lines'])['nrmse'].idxmin()]
        print(best_soft.loc[best_soft_params][
            ['missing_rate', 'n_lines', 'shrinkage_value', 'nrmse']].drop_duplicates())
        
        print("\nMelhores parâmetros para IterativeSVD:")
        best_svd = matrix_methods[matrix_methods['method'] == 'IterativeSVD']
        best_svd_params = best_svd.loc[best_svd.groupby(['missing_rate', 'n_lines'])['nrmse'].idxmin()]
        print(best_svd.loc[best_svd_params][
            ['missing_rate', 'n_lines', 'rank', 'nrmse']].drop_duplicates())
    
    # 4. Comparação KNN por número de vizinhos
    knn_results = df[df['method'] == 'KNNImputer']
    if not knn_results.empty:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='rank', y='nrmse', data=knn_results)
        plt.title('Desempenho do KNN por Número de Vizinhos')
        plt.ylabel('NRMSE')
        plt.xlabel('Número de Vizinhos (k)')
        plt.tight_layout()
        plt.savefig('knn_performance_by_neighbors.png')
        plt.close()
    
    # 5. Análise por dataset
    print("\n=== Desempenho por Dataset ===")
    print(df.groupby(['dataset_id', 'method'])['nrmse'].mean().unstack())
    
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='dataset_id', y='nrmse', hue='method', data=df)
    plt.title('Desempenho por Dataset')
    plt.ylabel('NRMSE')
    plt.xlabel('ID do Dataset')
    plt.tight_layout()
    plt.savefig('performance_by_dataset.png')
    plt.close()
    
    # 6. Identificação dos melhores métodos por cenário (CORRIGIDO)
    print("\n=== Melhores Métodos por Cenário ===")
    # Primeiro agrupamos e encontramos os índices dos melhores NRMSE
    best_indices = df.groupby(['dataset_id', 'missing_rate', 'n_lines'])['nrmse'].idxmin()
    # Depois usamos esses índices para selecionar as linhas
    best_methods = df.loc[best_indices][['dataset_id', 'missing_rate', 'n_lines', 'method', 'nrmse']]
    print(best_methods)
    
    # Contagem de vitórias por método
    print("\n=== Contagem de Melhores Desempenhos por Método ===")
    print(best_methods['method'].value_counts())

def main():
    # Carrega todos os resultados
    results_df = load_and_combine_results()
    
    # Limpeza e preparação dos dados
    results_df['n_lines'] = results_df['n_lines'].fillna(0)  # Para métodos não matriciais
    
    # Realiza a análise
    analyze_results(results_df)
    
    print("\nAnálise concluída! Gráficos salvos como arquivos PNG.")

if __name__ == "__main__":
    main()


=== Desempenho Médio por Método ===
                         rmse     nrmse
method                                 
KNNImputer           0.143166  0.032442
LinearInterpolation  0.149964  0.035150
SoftImpute           0.167498  0.035629
IterativeSVD         0.151200  0.043149

=== Desempenho por Taxa de Dados Faltantes ===
                         rmse                                   nrmse  \
missing_rate             10.0      20.0      30.0      40.0      10.0   
method                                                                  
IterativeSVD         0.142176  0.173529  0.158488  0.130608  0.052979   
KNNImputer           0.120941  0.173290  0.159382  0.119051  0.033024   
LinearInterpolation  0.117841  0.168051  0.161179  0.152785  0.032638   
SoftImpute           0.148564  0.192096  0.182124  0.147207  0.036868   

                                                   
missing_rate             20.0      30.0      40.0  
method                                             
Iterati

/tmp/ipykernel_5681/2680453862.py:39: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.barplot(x='method', y='nrmse', data=df, estimator=np.mean, ci=None)
/tmp/ipykernel_5681/2680453862.py:53: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.lineplot(x='missing_rate', y='nrmse', hue='method', data=df,



Melhores parâmetros para SoftImpute:


ValueError: Cannot index with multidimensional key

<Figure size 1200x800 with 0 Axes>